In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Data
jobs = [1, 2, 3, 4, 5]
time_A = [5, 1, 9, 3, 10]
time_B = [2, 6, 7, 8, 4]
n = len(jobs)

In [3]:
model = gp.Model('scheduling')

Restricted license - for non-production use only - expires 2025-11-24


In [14]:
x = {}
for i in range(n):
    for j in range(n):
        x[(i,j)] = model.addVar(0,1, vtype=GRB.INTEGER, name = 'x_%i_%i' %(i,j))

In [1]:
from gurobipy import Model, GRB, quicksum

# Processing time for Jobs on Machines A and B
jobs = [1, 2, 3, 4, 5]
time_A = [5, 1, 9, 3, 10]  # Time for Machine A
time_B = [2, 6, 7, 8, 4]   # Time for Machine B

n_jobs = len(jobs)

# Create a new model
m = Model("flow_shop_scheduling")

# Decision variables: X[i,j] = 1 if job i is at position j
X = m.addVars(n_jobs, n_jobs, vtype=GRB.BINARY, name="X")

# C_max: Completion time of the last job on Machine B (makespan)
C_max = m.addVar(vtype=GRB.CONTINUOUS, name="C_max")

# Constraints
# 1. Each job is assigned to exactly one position
for i in range(n_jobs):
    m.addConstr(quicksum(X[i,j] for j in range(n_jobs)) == 1, name=f"assign_job_{i}")

# 2. Each position is assigned to exactly one job
for j in range(n_jobs):
    m.addConstr(quicksum(X[i,j] for i in range(n_jobs)) == 1, name=f"assign_pos_{j}")

# 3. Completion time constraints for both machines
start_A = [m.addVar(vtype=GRB.CONTINUOUS, name=f"start_A_{j}") for j in range(n_jobs)]
start_B = [m.addVar(vtype=GRB.CONTINUOUS, name=f"start_B_{j}") for j in range(n_jobs)]

for j in range(n_jobs):
    # Start time of job j on Machine A
    m.addConstr(start_A[j] == quicksum(X[i,j] * (0 if j == 0 else start_A[j-1] + time_A[i]) for i in range(n_jobs)))

    # Start time of job j on Machine B depends on when job j finishes on Machine A and Machine B being free
    m.addConstr(start_B[j] == quicksum(X[i,j] * (start_A[j] + time_A[i]) for i in range(n_jobs)))

# 4. Define C_max as the completion time of the last job on Machine B
m.addConstr(C_max == quicksum(X[i,n_jobs-1] * (start_B[n_jobs-1] + time_B[i]) for i in range(n_jobs)))

# Objective: Minimize the makespan
m.setObjective(C_max, GRB.MINIMIZE)

# Optimize the model
m.optimize()

# Print the results
if m.status == GRB.OPTIMAL:
    print(f"Optimal schedule with makespan {C_max.X}")
    for i in range(n_jobs):
        for j in range(n_jobs):
            if X[i, j].X > 0.5:
                print(f"Job {jobs[i]} is scheduled at position {j+1}")


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 11 rows, 36 columns and 51 nonzeros
Model fingerprint: 0xbaaa490a
Model has 10 quadratic constraints
Variable types: 11 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.04s
Presolved: 129 rows, 94 columns, 403 nonzeros
Presolved model has 20 SOS constraint(s)
Variable types: 58 continuous, 36 integer (35 binary)
Found heuristic solution: objective 32.0000000
Found heuristic solution: objec